In [1]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch import nn
from torchsummary import summary

In [2]:
class Email(Dataset):
    
    def __init__(self, file_path):
        self.file_path = pd.read_csv(file_path)
        
    def __len__(self):
        return len(self.file_path)
    
    def __getitem__(self, index):
        word_count = torch.tensor([self.file_path.iloc[index, 1:-1]], dtype=torch.float)
        label = torch.tensor([self.file_path.iloc[index, -1]], dtype=torch.float)
        return word_count, label

In [3]:
class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3000, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x

In [4]:
def train_single_epoch(model, data_loader, loss_fn, optimiser):

    for sample, target in data_loader:

        # calculate loss
        target = target.flatten()
        prediction = model(sample)
        prediction = prediction.flatten()
        loss = loss_fn(prediction, target)

        # backpropagate error and update weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    return loss.item()

In [5]:
em_set = Email("emails.csv")
print("length of the dataset is:", len(em_set))

length of the dataset is: 5172


In [6]:
rate = 0.8
LEN_em = len(em_set)
LEN_train = int(rate * LEN_em)
LEN_test = LEN_em - LEN_train

In [7]:
train_data, test_data = random_split(em_set, [LEN_train, LEN_test])
train_loader = DataLoader(train_data, batch_size = 128, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 128, shuffle = False)

In [8]:
model = Network()
summary(model, (1, 3000))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 1]           3,001
           Sigmoid-2                 [-1, 1, 1]               0
Total params: 3,001
Trainable params: 3,001
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.02
----------------------------------------------------------------


In [9]:
loss_fn = nn.MSELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
EPOCHS = 10
for i in range(EPOCHS):
    print(f"Epoch {i+1}")
    loss = train_single_epoch(model, train_loader, loss_fn, optimiser)
    print(f"training loss: {loss}")
    print("---------------------------")
print("Finished training")

Epoch 1
training loss: 0.10665984451770782
---------------------------
Epoch 2
training loss: 0.060369499027729034
---------------------------
Epoch 3
training loss: 0.05202246829867363
---------------------------
Epoch 4
training loss: 0.04386045038700104
---------------------------
Epoch 5
training loss: 0.03767269477248192
---------------------------
Epoch 6
training loss: 0.03429638594388962
---------------------------
Epoch 7
training loss: 0.03956405818462372
---------------------------
Epoch 8
training loss: 0.01957833021879196
---------------------------
Epoch 9
training loss: 0.02282893657684326
---------------------------
Epoch 10
training loss: 0.02934592217206955
---------------------------
Finished training


In [11]:
correct_sample = 0.0
num_sample = 0.0

model.eval()
with torch.no_grad():
    for sample, target in test_loader:
        
        target = target.flatten()
        prediction = model(sample)
        prediction = prediction.flatten()
        y_pred = torch.round(prediction)
        num_sample += len(target)
        correct_sample += (y_pred == target).sum().float()
        
    acc = correct_sample / num_sample
    acc = acc.item()

print(f"validation accuracy: {acc}")

validation accuracy: 0.9748792052268982


# Save the model

In [12]:
torch.save(model.state_dict(), "spam-pytorch-nn.pth")

# Load the pretrained model

In [13]:
imported_model = Network()
state_dict = torch.load("spam-pytorch-nn.pth")
imported_model.load_state_dict(state_dict)

<All keys matched successfully>

---

# Define your email

In [14]:
my_mail = "this is the first email"

In [15]:
# get the key words
df = pd.read_csv("emails.csv")

column_names = []
for column in df.columns:
    column_names.append(column)
    
del column_names[0]
del column_names[-1]

# Calculate the occurrences

In [16]:
from MyClass import Mail

mail1 = Mail(column_names, my_mail)
num1 = mail1.get_occurrences()

# Spam or Non-spam

In [17]:
if imported_model(torch.tensor(num1, dtype=torch.float)) >= 0.5:
    print("This is a spam")
else:
    print("This is a normal email")

This is a normal email
